In [1]:
### for emotion detection
https://github.com/monologg/GoEmotions-pytorch
### For toxicity detection
perspective api
### For politeness detection
https://github.com/AlafateABULIMITI/politeness-detection
### For language quality
https://github.com/WanzhengZhu/GPS/blob/master/language_quality.py
### Other metrics are aleady added add two metric from the GPS model.
### Implement GPS model as a baseline.
save_path='../HULK_new/Counterspeech/Saved_Models'

SyntaxError: invalid syntax (<ipython-input-1-92ae64a6dc1e>, line 2)

In [1]:
import json
import time
from tqdm.notebook import tqdm

False

In [3]:
import sys
sys.path.append("../Generation")
sys.path.append('../Generation/eval')

In [4]:

import glob
import json
from loguru import logger
import pandas as pd 
from eval import *
path_datasets = '/home/adarsh-binny' + '/HULK_new/Counterspeech/Datasets'
path_result   = '/home/adarsh-binny' + '/HULK_new/Counterspeech/Results_new'
path_result1   = '/home/adarsh-binny' + '/HULK_new/Counterspeech/Results'
save_path     = '/home/adarsh-binny' + '/HULK_new/Counterspeech/metrics_results/'

In [5]:
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
from pprint import pprint

cache_dir='/home/adarsh-binny/HULK_new/Saved_models'

tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-ekman",cache_dir=cache_dir)
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-ekman",cache_dir=cache_dir)

In [6]:

import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from torch import cuda

VALID_BATCH_SIZE = 16

In [7]:
device = "cuda" if cuda.is_available() else "cpu"

In [8]:
model.to(device);

In [9]:

class PDataset(Dataset):
    def __init__(self, hypo, tokenizer, max_len=200):
        self.len = len(hypo)
        self.data = hypo
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        text = self.data[index]
        #print(text)
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

In [10]:
    
def goemotion(hypo):
    N =len(hypo)
    print("Length of Hypothesis: ", N)
    hypo_ = PDataset(hypo, tokenizer, 200)
    testing_loader = DataLoader(hypo_, **test_params)
    model.eval()
    tr_loss = 0
    n_correct = 0 
    n_wrong = 0
    total = {"joy":0, "fear":0, "anger":0, "love":0, "surprise":0, "sadness":0};
    n1 = N//8
    if N%VALID_BATCH_SIZE!=0:
        n1+=1
    
    with torch.no_grad():
        with tqdm(n1) as pbar:
            for _, data in enumerate(testing_loader, 0):
                #print(data)
                ids = data['ids'].to(device, dtype = torch.long)
                mask = data['mask'].to(device, dtype = torch.long)
                outputs = model(ids, mask)[0]
                #print(outputs.shape)
                #print(outputs)
                outputs = torch.nn.Sigmoid()(outputs).to('cpu').numpy()
                #print(outputs)
                for scores in outputs:
                    #print(scores)
                    for idx, s in enumerate(scores):
                        if model.config.id2label[idx] in ["joy", "fear", "anger", "love", "surprise", "sadness"]:
                            total[model.config.id2label[idx]]+=s
                pbar.update()
    for i in total.keys():
        total[i]/=N
    return total
    

In [11]:
texts = [
    
    "Hey that's a thought! Maybe we need [NAME] to be the celebrity vaccine endorsement!",
    "it’s happened before?! love my hometown of beautiful new ken 😂😂",
    "I love you, brother.",
    "Troll, bro. They know they're saying stupid shit. The motherfucker does nothing but stink up libertarian subs talking shit",
]

goemotion(texts)

0it [00:00, ?it/s]

Length of Hypothesis:  4


/home/adarsh-binny/.conda/envs/csgen/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
1it [00:00,  4.58it/s]


{'joy': 0.5809454414993525,
 'fear': 0.0017176893597934395,
 'anger': 0.1949171123560518,
 'love': 0.0,
 'surprise': 0.1574256321764551,
 'sadness': 0.009181847679428756}

In [12]:
def calc_emotion(hypo, emotion, normal=False):
    scores = goemotion(hypo)
    #print(scores)
    if normal==False:
        return scores[emotion]
    else:
        return scores

In [13]:
texts = [
    
    "Hey that's a thought! Maybe we need [NAME] to be the celebrity vaccine endorsement!",
    "it’s happened before?! love my hometown of beautiful new ken 😂😂",
    "I love you, brother.",
    "Troll, bro. They know they're saying stupid shit. The motherfucker does nothing but stink up libertarian subs talking shit",
]

calc_emotion(texts, "anger", normal=False)

1it [00:00, 25.97it/s]

Length of Hypothesis:  4


0.1949171123560518

In [13]:
def metrics(trained_on,tested_on):
    train_path = glob.glob(path_datasets+'/*'+trained_on+'*/*rain*')[0]
    gen        = glob.glob(path_result+'/*'+trained_on+'*'+tested_on+'*1628*')
    ref        = glob.glob(path_result1+'/*'+tested_on+'*'+'references'+'*')[0]
    
#     print(gen,ref)
    with open(ref, 'r') as file:
        ref_dict   = json.loads(file.read())
        
    scores = {}
    for files in gen:
        with open(files, 'r') as file:
            gen_dict  = json.loads(file.read())
        emotion = gen_dict['params']['task_name'][0][1]
        #print(files)
        #print(emotion)
        if ("Emotion" in gen_dict['params']['task_name'][0][0] or "gedi" not in files)==False:
            continue
        print(files)
        gpu_id    = gen_dict['params']['gpu_id']
        hypo = []
        refs = []
        signature = files.split("_")[-1].split(".")[0]
        for key in gen_dict['samples']:
            for sentences in gen_dict['samples'][key]['counterspeech_model']:
                hypo.append(sentences)
                refs.append(ref_dict['samples'][key]['counterspeech_model'])
        
        train = pd.read_csv(train_path)
        train_set = list(zip(train['initiator_message'].tolist(),train['reply_message'].tolist()))
        params = hypo
        if "gedi" not in files:
            average_scores = calc_emotion(hypo, "", normal=True)
            key = trained_on+'_'+tested_on+'_'+emotion+'_'+signature+'_'+str(gpu_id)
            scores[key] = average_scores
            print("Scores:\n",average_scores)
            print()
        else:
            print(emotion)
            average_score = calc_emotion(hypo, emotion)
            key = trained_on+'_'+tested_on+'_'+emotion+'_'+signature+'_'+str(gpu_id)
            scores[key] = average_score
            print(emotion, "Score:",average_score)
            print()
    json.dump(scores, open(save_path +tested_on+'emotion.json','w'),indent = 4)    

In [44]:
def metrics(trained_on,tested_on):
    train_path = glob.glob(path_datasets+'/*'+trained_on+'*/*rain*')[0]
    gen        = glob.glob(path_result+'/*'+trained_on+'*'+tested_on+'*16*_multi*')
    ref        = glob.glob(path_result1+'/*'+tested_on+'*'+'references'+'*')[0]
    
#     print(gen,ref)
    with open(ref, 'r') as file:
        ref_dict   = json.loads(file.read())
        
    scores = {}
    for files in gen:
        with open(files, 'r') as file:
            gen_dict  = json.loads(file.read())
        emotion = gen_dict['params']['task_name'][0][1]
        #print(files)
        emotion = "fear"
        print(gen_dict['params']['task_name'])
        print(gen_dict['params']['disc_weight'])
        #print(emotion)
        #if ("Emotion" in gen_dict['params']['task_name'][0][0] or "gedi" not in files)==False:
        #    continue
        if [['Toxicity', 'toxic'], ['Politeness', 'polite']]!=gen_dict['params']['task_name']:
            continue
        
        print(files)
        gpu_id    = gen_dict['params']['gpu_id']
        hypo = []
        refs = []
        signature = files.split("_")[-1].split(".")[0]
        for key in gen_dict['samples']:
            for sentences in gen_dict['samples'][key]['counterspeech_model']:
                hypo.append(sentences)
                refs.append(ref_dict['samples'][key]['counterspeech_model'])
        
        train = pd.read_csv(train_path)
        train_set = list(zip(train['initiator_message'].tolist(),train['reply_message'].tolist()))
        params = hypo
        if "gedi" not in files:
            average_scores = calc_emotion(hypo, "", normal=True)
            key = trained_on+'_'+tested_on+'_'+emotion+'_'+signature+'_'+str(gpu_id)
            scores[key] = average_scores
            print("Scores:\n",average_scores)
            print()
        else:
            print(emotion)
            average_score = calc_emotion(hypo, emotion)
            key = trained_on+'_'+tested_on+'_'+emotion+'_'+signature+'_'+str(gpu_id)
            scores[key] = average_score
            print(emotion, "Score:",average_score)
            print()
    #json.dump(scores, open(save_path +tested_on+'emotion.json','w'),indent = 4)    

****Reddit****

In [45]:
metrics('CONAN','CONAN')

0it [00:00, ?it/s]

[['Emotion', 'sadness'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'anger'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'fear'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'anger'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'joy'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'fear'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'joy'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'sadness'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'fear'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'anger'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Toxicity', 'toxic'], ['Politeness', 'polite']]
[0.5, 0.5]
/home/adarsh-binny/HULK_new/Counterspeech/Results_new/Generator-CONAN_DialoGPT-medium_on_CONAN_gedi_huggingface_1629965093_multi.json
fear
Length of Hypothesis:  1120


70it [00:14,  4.93it/s]

fear Score: 0.009005626851675125

[['Emotion', 'joy'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'sadness'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'fear'], ['Politeness', 'polite']]
[0.4, 0.3, 0.3]


In [46]:
metrics('Reddit','Reddit')

[['Emotion', 'fear'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'anger'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'anger'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'sadness'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'joy'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Toxicity', 'toxic'], ['Politeness', 'polite']]
[0.5, 0.5]
/home/adarsh-binny/HULK_new/Counterspeech/Results_new/Generator-Reddit_DialoGPT-medium_on_Reddit_gedi_huggingface_1629982885_multi.json


1it [00:00,  5.78it/s]

fear
Length of Hypothesis:  6495


406it [01:48,  3.74it/s]


fear Score: 0.0021131527654708704

[['Emotion', 'joy'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'anger'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'joy'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'fear'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'sadness'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'sadness'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'fear'], ['Toxicity', 'toxic']]
[0.5, 0.5]


In [47]:
metrics('Gab', 'Gab')

[['Emotion', 'fear'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'joy'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'sadness'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'joy'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'sadness'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'fear'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'anger'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Emotion', 'anger'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
[['Emotion', 'anger'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'joy'], ['Politeness', 'polite']]
[0.5, 0.5]
[['Toxicity', 'toxic'], ['Politeness', 'polite']]
[0.5, 0.5]
/home/adarsh-binny/HULK_new/Counterspeech/Results_new/Generator-Gab_DialoGPT-medium_on_Gab_gedi_huggingface_1630031392_multi.json


0it [00:00, ?it/s]

fear
Length of Hypothesis:  18115


1133it [07:32,  2.50it/s]


fear Score: 0.002000224312458338

[['Emotion', 'sadness'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'fear'], ['Toxicity', 'toxic']]
[0.5, 0.5]
[['Emotion', 'sadness'], ['Politeness', 'polite'], ['Toxicity', 'toxic']]
[0.4, 0.3, 0.3]
